In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
train_data=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [3]:
train_df=pd.DataFrame(train_data)
test_df=pd.DataFrame(test_data)

X_train=train_df.drop(['ShoppingMall','FoodCourt','RoomService','Spa', 'Name','VRDeck','Cabin','Transported','PassengerId'],axis=1)
X_train['Destination'].replace(['TRAPPIST-1e','PSO J318.5-22','55 Cancri e'], [0,1,2],inplace=True)
X_train['HomePlanet'].replace(['Europa','Earth','Mars'],[0,1,2],inplace=True)
y_train=train_df['Transported']
X_train=X_train.astype({"CryoSleep":bool,"VIP":bool})

In [4]:
print(X_train)
print(y_train)

      HomePlanet  CryoSleep  Destination   Age    VIP
0            0.0      False          0.0  39.0  False
1            1.0      False          0.0  24.0  False
2            0.0      False          0.0  58.0   True
3            0.0      False          0.0  33.0  False
4            1.0      False          0.0  16.0  False
...          ...        ...          ...   ...    ...
8688         0.0      False          2.0  41.0   True
8689         1.0       True          1.0  18.0  False
8690         1.0      False          0.0  26.0  False
8691         0.0      False          2.0  32.0  False
8692         0.0      False          0.0  44.0  False

[8693 rows x 5 columns]
0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool


In [5]:
X_test=test_df.drop(['ShoppingMall','FoodCourt','RoomService','Spa', 'Name','VRDeck','Cabin','PassengerId'],axis=1)
X_test['HomePlanet'].replace(['Europa','Earth','Mars'],[0,1,2],inplace=True)
X_test['Destination'].replace(['TRAPPIST-1e','PSO J318.5-22','55 Cancri e'], [0,1,2],inplace=True)
X_test=X_test.astype({"CryoSleep":bool,"VIP":bool})
print(X_test)

      HomePlanet  CryoSleep  Destination   Age    VIP
0            1.0       True          0.0  27.0  False
1            1.0      False          0.0  19.0  False
2            0.0       True          2.0  31.0  False
3            0.0      False          0.0  38.0  False
4            1.0      False          0.0  20.0  False
...          ...        ...          ...   ...    ...
4272         1.0       True          0.0  34.0  False
4273         1.0      False          0.0  42.0  False
4274         2.0       True          2.0   NaN  False
4275         0.0      False          NaN   NaN  False
4276         1.0       True          1.0  43.0  False

[4277 rows x 5 columns]


In [6]:
my_imputer = SimpleImputer()
X_train= my_imputer.fit_transform(X_train)
X_test=my_imputer.fit_transform(X_test)

In [7]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

LogisticRegression()

In [8]:
y_pred = logistic_model.predict(X_test)
print(y_pred)

[ True False  True ...  True False  True]


In [9]:
submission=test_df[["PassengerId"]]
print(submission)

     PassengerId
0        0013_01
1        0018_01
2        0019_01
3        0021_01
4        0023_01
...          ...
4272     9266_02
4273     9269_01
4274     9271_01
4275     9273_01
4276     9277_01

[4277 rows x 1 columns]


In [10]:
submission["Transported"]=y_pred
print(submission)

     PassengerId  Transported
0        0013_01         True
1        0018_01        False
2        0019_01         True
3        0021_01        False
4        0023_01        False
...          ...          ...
4272     9266_02         True
4273     9269_01        False
4274     9271_01         True
4275     9273_01        False
4276     9277_01         True

[4277 rows x 2 columns]


/tmp/ipykernel_20/644117842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission["Transported"]=y_pred


In [11]:
submission.to_csv("submission_3.csv",index=None)